In [ ]:
!pip install transformers==4.37.2 optimum==1.12.0 --quiet
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --quiet
!pip install langchain==0.1.9 --quiet
# !pip install chromadb
!pip install sentence_transformers==2.4.0 --quiet
!pip install unstructured --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six==20221105 --quiet
!pip install unstructured-inference --quiet
!pip install faiss-gpu==1.7.2 --quiet
!pip install pikepdf==8.13.0 --quiet
!pip install pypdf==4.0.2 --quiet
!pip install pillow_heif==0.15.0 --quiet

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

model_name = "TheBloke/Llama-2-13b-Chat-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

gen_cfg = GenerationConfig.from_pretrained(model_name)
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 # 0.0
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=gen_cfg
)

llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
from textwrap import fill
from langchain.prompts import PromptTemplate

template = """
[INST] <>
You are an AI assistant. You are truthful, unbiased and honest in your response.

If you are unsure about an answer, truthfully say "I don't know"
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

#text = "Explain artificial intelligence in a few lines"
#result = llm.invoke(prompt.format(text=text))
#print(fill(result.strip(), width=100))
text = "Explain artificial intelligence in a few lines"
formatted_prompt = prompt.format(text=text)
result = llm.invoke(formatted_prompt)

# Extract the response text after the closing [/INST] tag
response_start = result.find("[/INST]") + len("[/INST]")
response = result[response_start:].strip()

print(fill(response, width=100))

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores.utils import filter_complex_metadata # 'filter_complex_metadata' removes complex metadata that are not in str, int, float or bool format

pdf_loader = UnstructuredPDFLoader("/content/The 2024 ICC Men's T20 World Cup.pdf")
pdf_doc = pdf_loader.load()
updated_pdf_doc = filter_complex_metadata(pdf_doc)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_pdf_doc = text_splitter.split_documents(updated_pdf_doc)
len(chunked_pdf_doc)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
%%time
# Create the vectorized db with FAISS
from langchain.vectorstores import FAISS
db_pdf = FAISS.from_documents(chunked_pdf_doc, embeddings)

# Create the vectorized db with Chroma
# from langchain.vectorstores import Chroma
# db_pdf = Chroma.from_documents(chunked_pdf_doc, embeddings)

In [ ]:
%%time
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# use the recommended propt style for the LLAMA 2 LLM
prompt_template = """
[INST] <>
Use the following context to Answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.

<>

{context}

Question: {question} [/INST]
"""

#prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # retriever=db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 5, 'score_threshold': 0.8})
    # Similarity Search is the default way to retrieve documents relevant to a query, but we can use MMR by setting search_type = "mmr"
    # k defines how many documents are returned; defaults to 4.
    # score_threshold allows to set a minimum relevance for documents returned by the retriever, if we are using the "similarity_score_threshold" search type.
    # return_source_documents=True, # Optional parameter, returns the source documents used to answer the question
    retriever=db_pdf.as_retriever(), # (search_kwargs={'k': 5, 'score_threshold': 0.8}),
    chain_type_kwargs={"prompt": prompt},
)
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_pdf.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
query = "which team is announced as team of tournament by ICC in t20 world cup 2024?"
result = Chain_pdf({"query": query})

# Extract the response text after the closing [/INST] tag
response_start = result['result'].find("[/INST]") + len("[/INST]")
response = result['result'][response_start:].strip()

print(fill(response, width=100))

In [ ]:
!pip install pyngrok

In [ ]:
pip install flask-cors


In [ ]:
import os
from flask import Flask, request, jsonify
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pyngrok import ngrok
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# Define your prompt template and retrieval chain once
prompt_template = """
[INST] <>
Use the following context to answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.

<>

{context}

Question: {question} [/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_pdf.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)

@app.route('/ask', methods=['POST'])
def ask():
    data = request.json
    query = data.get("query")
    result = Chain_pdf({"query": query})

    response_start = result['result'].find("[/INST]") + len("[/INST]")
    response = result['result'][response_start:].strip()

    return jsonify({"response": response})

if __name__ == '__main__':
    port = 5000

    # Set ngrok authtoken
    ngrok.set_auth_token("2iC5RBSZnGE9oAi1S9AJHFg4cxE_4EQbdAH37Yn8LhCZ2hzGc")

    public_url = ngrok.connect(port)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
    app.run(port=port)
